In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from core.pandas_utils import series_start_end_diff
from  transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries, ProcessedTimeSeries
from transform.raw_results.config import *
from transform.processed_results.main import make_soh_presentable_per_vehicle
from core.caching_utils import cache_result

from transform.raw_results.mercedes_results import *

Notebook pour forcer la décroissance des SoH 

## Import data

In [ ]:
@cache_result("../tesla/tesla_results.parquet", on="local_storage")
def get_results():
    results = (
        TeslaProcessedTimeSeries("tesla", force_update=False)
        .query("trimmed_in_charge")
        .groupby(["vin", "trimmed_in_charge_idx"])
        .agg(
            #energy_added=pd.NamedAgg("charge_energy_added", series_start_end_diff),
            energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
            energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            charging_power=pd.NamedAgg("charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
        )
        .reset_index(drop=False)
        .eval("energy_added = energy_added_end - energy_added_min")
        .eval("soh = energy_added / (soc_diff / 100.0 * capacity)")
        .query("soc_diff > 40 & soh.between(0.75, 1.05)")
        .eval("level_1 = soc_diff * (charging_power < @LEVEL_1_MAX_POWER) / 100")
        .eval("level_2 = soc_diff * (charging_power.between(@LEVEL_1_MAX_POWER, @LEVEL_2_MAX_POWER)) / 100")
        .eval("level_3 = soc_diff * (charging_power > @LEVEL_2_MAX_POWER) / 100")
	.eval("bottom_soh = soh.between(0.75, 0.9)")
        .eval("fixed_soh_min_end = soh.mask(tesla_code == 'MTY13', soh / 0.96)")
        .eval("fixed_soh_min_end = fixed_soh_min_end.mask(bottom_soh & tesla_code == 'MTY13', fixed_soh_min_end + 0.08)")
        .eval("soh = fixed_soh_min_end")
        .sort_values(["tesla_code", "vin", "date"])
    )
    return results

df = get_results()

## Aplliquer une décroissance après le traitement

In [ ]:
def calculate_range_diff(df: pd.DataFrame) -> pd.DataFrame:
    """Calcule la différence de range entre de l'odometre pour la dernière valeur de SoH qui se répète.
    
    Args:
        df (pd.DataFrame): Le DataFrame contenant les données avec la colonne 'soh' et 'odometer'.

    Returns:
        pd.DataFrame: Une ligne contenant le 'soh', 'range_diff' 'last', 'first' (pour la colonne 'odometer').
    """
    grouped = df.groupby('soh')['odometer'].agg(['first', 'last', 'count'])
    grouped['range_diff'] = grouped['last'] - grouped['first']
    # filtrage pour n'avoir les lignes où 'first' est différent de 'last'
    result_df = grouped[grouped['first'] != grouped['last']].reset_index()
    if len(result_df) > 0:
        return result_df.sort_values("soh").loc[0]
    return None

In [ ]:
def apply_soh_decay(df: pd.DataFrame) -> pd.DataFrame:
    """Applique une décroissance sur la colonne 'soh' d'un DataFrame en fonction du kilométrage 
    ('odometer') et des charges. La décroissance est calculée selon une formule exponentielle 
    et appliquée de manière vectorisée.

    Args:
        df (pd.DataFrame): Un DataFrame contenant avec les colonnes 'soh', 'odometer', et 'charges'.

    Returns:
        pd.DataFrame: Le DataFrame avec la colonne 'soh' mise à jour.
    """
    
    identified = calculate_range_diff(df)
    
    # check si la valeur maximale de 'odometer' n'est pas égale à 'last' de la ligne identifiée
    if identified.empty or (df['odometer'].max() != identified['last']) or  (identified['range_diff'] < 5_000):
        return df['soh']
    
    # Définition de la fonction de calcul pour la décroissance de 'SoH en fonction du kilométrage (km) et des charges
    def formula(soh:float, km:float|int, charges:int) -> float:
        """ Formule de décroissance sur la valeur 'soh' en fonction du kilométrage et du nombre de charges.

        Args:
            soh (float): La valeur de SoH de départ.
            km (float | int):le nombre de kms
            charges (int): Le nombre de charges

        Returns:
            float: La nouvelle valeur de SoH
        """
        
        lambda_km = np.log(0.92) / 200_000  
        lambda_N = np.log(0.92) / 4000      
        soh = soh * np.exp(lambda_km * km) * np.exp(lambda_N * charges)
        return soh
    
    value_decrease = formula(identified['soh'], identified['range_diff'], identified['count'])
    
    df_copy = df.copy()

    # Filtrer les lignes où 'soh' est égal à la valeur minimale de 'soh'
    mask = df_copy['soh'] == df_copy['soh'].min()
    df_repeated = df_copy[mask]
    
    # get first and last index 
    first_indices = df_repeated.groupby('soh').head(1).index
    last_indices = df_repeated.groupby('soh').tail(1).index
    
    soh_new_values = df_copy['soh'].copy()
    soh_new_values.loc[last_indices] = value_decrease

    first_values = df_copy.loc[first_indices, 'soh'].values
    last_values = value_decrease  
    lengths = df_repeated.groupby('soh').size().values 

    new_soh_values = np.concatenate([np.linspace(f, l, n) for f, l, n in zip([first_values], [last_values], lengths)])

    soh_new_values.loc[df_repeated.index] = new_soh_values.reshape(new_soh_values.shape[0],)

    return soh_new_values


In [ ]:
# LRW3E7FA4LC103400 -> vin pour voir appliqué
# XP7YGCES7RB428560 -> vin pour lequel on n'effectue rien
df_vin = df[df['vin']=="LRW3E7FA4LC103400"].copy()

soh_prod = make_soh_presentable_per_vehicle(df_vin)
soh_prod['soh_update'] = apply_soh_decay(soh_prod[['soh', 'odometer']])

soh_prod[['soh','soh_update', 'odometer']]


## Appliquer un decroissance avec un calcul de windows fonction 

Remplace ce qui est fait aujourd'hui avec la fonction `force_monotonic_decrease`

In [ ]:
def force_decay(df):
    rolling_mean = df["soh"].rolling(window=3, min_periods=1).mean()

    # Forcer la monotonie strictement décroissante 
    rolling_decreasing = rolling_mean.cummin()
    soh_decreasing = rolling_decreasing  - (df["odometer"] - df["odometer"].min()) * 1e-8
    return soh_decreasing

In [ ]:
df_vin['rolling_soh'] = force_decay(df_vin)

In [ ]:
df_vin[['odometer', 'soh', 'rolling_soh']]

## Plot 

In [ ]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=soh_prod['odometer'], y=soh_prod['soh'], mode="markers", opacity=1, name='Before'), )
fig.add_trace(go.Scatter(x=soh_prod['odometer'], y=soh_prod['soh_update'], mode="markers", opacity=.4, name='After'), )
fig.add_trace(go.Scatter(x=df_vin['odometer'], y=df_vin['soh'], mode="markers", opacity=1, name='get result'), )
fig.add_trace(go.Scatter(x=df_vin['odometer'], y=df_vin['rolling_soh'], mode="markers", opacity=1, name='rolling'), )
